# OptiGuide Example with Pyomo


Authors: [Phuc Van Phan](https://github.com/pphuc25), [Alfredo Hernandez](https://github.com/alfredoihernandez), [Beibin Li](https://github.com/beibinli)



For this notebook, we mainly introduce how to use Pyomo with OptiGuide, and we skipped the overview of OptiGuide what-if analysis. If you haven't checked the [OptiGuide example with Gurobi](optiguide_example.ipynb), please check it first.


In [1]:
# Install Required Packages
# %pip install optiguide

In [2]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import autogen
import autogen
from autogen.agentchat import Agent, UserProxyAgent
from optiguide import OptiGuideAgent

In [3]:
config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
        }
    }
)

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [4]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/benchmark/application/coffee_pyomo.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open("../benchmark/application/coffee_pyomo.py", "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time

from pyomo.environ import (Binary, ConcreteModel, Constraint, Integers,
                           Objective, SolverFactory, SolverStatus,
                           TerminationCondition, Var, minimize)

# Example data
capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}
shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
.
.
.

# You can change the solver as per your requirement
m = solver.solve(model)

print(time.ctime())
if m.solver.termination_condition == TerminationCondition.optimal:
    print(f'Optimal cost: {model.obj()}')
else:
    print("Not solved to optimality. Optimization status:", m.solver.termination_condition)




In [5]:
# In-context learning examples.
pyomo_example_qa = """
----------
Question: Why is it not recommended to use just one supplier for roastery 2?
Answer Code:
```python
m.z = Var(suppliers, domain=Binary, bounds=(0, None))
def one_supplier_constraint(m):
    return sum(m.z[s] for s in suppliers) <= 1
m.OneSupplierConstraint = Constraint(rule=one_supplier_constraint)
def roastery_2_demand_met_constraint(m, s):
    return m.x[s,'roastery2'] <= capacity_in_supplier[s] * m.z[s]
m.Roastery2DemandMetConstraint = Constraint(suppliers, rule=roastery_2_demand_met_constraint)
```

----------
Question: What if there's a 13% jump in the demand for light coffee at cafe1?
Answer Code:
```python
light_coffee_needed_for_cafe["cafe1"] = light_coffee_needed_for_cafe["cafe1"] * (1 + 13/100)
```
"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [6]:
agent = OptiGuideAgent(
    name="OptiGuide Coffee Example",
    source_code=code,
    debug_times=5,
    example_qa=pyomo_example_qa,
    solver_software="pyomo",
    use_safeguard=True,
    llm_config={
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

Fri Jan 19 10:30:43 2024
Optimal cost: 2470.0


Now, let's create a user's question.

In [7]:
# user.initiate_chat(agent, message="What if we prohibit shipping from supplier 1 to roastery 2?")
user.initiate_chat(agent, message="Why is it not recommended to use just one supplier for roastery 2?")

user (to OptiGuide Coffee Example):

Why is it not recommended to use just one supplier for roastery 2?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Adding constraint to ensure roastery 2 does not use just one supplier
m.z = Var(suppliers, domain=Binary)
def one_supplier_constraint(m):
    return sum(m.z[s] for s in suppliers) <= 1
m.OneSupplierConstraint = Constraint(rule=one_supplier_constraint)

def roastery_2_demand_met_constraint(m, s):
    return m.x[s, 'roastery2'] <= capacity_in_supplier[s] * m.z[s]
m.Roastery2DemandMetConstraint = Constraint(suppliers, rule=roastery_2_demand_met_constraint)
```

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to safeguard):


--- Code ---
# Adding constraint to ensu

In [9]:
user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")

user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Reducing the capacity of supplier1 to half of its current value
capacity_in_supplier['supplier1'] = capacity_in_supplier['supplier1'] / 2
```
This snippet modifies the capacity of supplier1 in the `capacity_in_supplier` dictionary by halving its current value before the constraints are added to the model that uses the capacities. It will directly influence the supply constraint for supplier1 in the optimization model.

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to safeguard):


--- Code ---
# Reducing the capacity of supplier1 to half o